In [1]:
require 'sequel'
DB = Sequel.connect(adapter: 'postgres',
                    host: 'postgres', 
                    user: 'postgres',
                    password: 'mysecretpassword',
                    database: 'adr',
                    search_path: ['mpd', 'meddra','reports'])

#<Sequel::Postgres::Database: {:adapter=>"postgres", :host=>"postgres", :user=>"postgres", :password=>"mysecretpassword", :database=>"adr", :search_path=>["mpd", "meddra", "reports"]}>

## Model

In [40]:


Object.send(:remove_const, :Drug) if defined?(Drug)
Object.send(:remove_const, :Report) if defined?(Report)
Object.send(:remove_const, :Adr) if defined?(Adr)
Object.send(:remove_const, :CommercialDrug) if defined?(CommercialDrug)
Object.send(:remove_const, :DrugAdministration) if defined?(DrugAdministration)
Object.send(:remove_const, :AtcClass) if defined?(AtcClass)



class Drug < Sequel::Model(DB[:pharmaceutical_products])
  set_primary_key [:id]
  one_to_many :commercial_drugs, key: :pharmaceutical_product_id
end 

class Report < Sequel::Model(DB)
  
  set_primary_key [:id]
  one_to_many :drug_administrations
  many_to_many :adrs, join_table: :meddra_adrs, left_key: :report_id,
  right_key: :meddra_low_level_term_id
  many_to_many :commercial_drugs, join_table: :treatments, left_key: :report_id, right_key: :mpd_medicinal_product_id
  
end 

class Adr < Sequel::Model(DB[:low_level_terms])
  set_primary_key [:id]
  many_to_many :reports, join_table: :meddra_adrs, left_key: :meddra_low_level_term_id ,
  right_key: :report_id
  
  def drug_count(params = {})
    
    default_params = {
      include_nil_dates: true,
      start_date: Date.parse("1014-11-17"), 
      end_date: Date.today, 
      atc_prefix: ""}
    
    p = default_params.merge(params)
    
    selected_reports = reports.select do |report| 
      if report.fill_date.nil?
        p[:include_nil_dates]
      else  
        p[:start_date] <=  report.fill_date && 
        report.fill_date <= p[:end_date]
      end  
    end #array of reports
    
    report_drugs = selected_reports.map do |report|
      report.commercial_drugs
    end  # array of arrays of commercial_drugs
    
    adr_drugs =  report_drugs.flatten #array of commercial_drugs
    
    adr_selected_drugs = adr_drugs.select do |drug|
      (not drug.nil?) && drug.atc_code.start_with?(p[:atc_prefix])
    end  #array of commercial_drugs
        
    adr_selected_drugs.count_occurrences
    
  end  

end 

class CommercialDrug < Sequel::Model(DB[:medicinal_products])
  set_primary_key [:id]
  one_to_one :drug, primary_key: :pharmaceutical_product_id, key: :id
  one_to_many :drug_administrations, primary_key: :id, key: :mpd_medicinal_product_id
  one_to_one :atc_class, primary_key: :atc_class_id, key: :id
  
  def atc_code
    atc_class.code
  end  
  
end  

class AtcClass< Sequel::Model(DB)
end  

class DrugAdministration < Sequel::Model(DB[:treatments])
  set_primary_key [:id]
  one_to_one :commercial_drug,  primary_key: :mpd_medicinal_product_id, key: :id
  one_to_one :report,  primary_key: :report_id, key: :id
  
  def adr_count
     ## EXERCISE
  end  
    
end 


class Array
  
  def count_occurrences
    r = {}
    group_by{|item| item}.each do |item, copies|
      r[item] = copies.length
    end  
    r
  end  
  
end 

:count_occurrences

In [48]:
Adr[10030109].drug_count.each{|cdrug, count| puts [count,cdrug.atc_code,cdrug.drug.name, cdrug.name]  }

[1, "C09BA05", "RAMIPRIL/IDROCLOROTIAZIDE", "TRIATEC HCT*14 cpr 5 mg + 25 mg"]
[1, "B01AX05", "FONDAPARINUX SODICO", "ARIXTRA*10 siringhe SC 2,5 mg 0,5 ml"]
[4, "B01AA03", "WARFARIN SODICO", "COUMADIN*30 cpr div 5 mg"]
[1, "C10AA05", "ATORVASTATINA CALCIO TRIIDRATO", "TORVAST*10 cpr riv 10 mg"]
[1, "A10BD07", "SITAGLIPTIN FOSFATO MONOIDRATO/METFORMINA CLORIDRATO", "JANUMET*56 cpr riv 50 mg + 1.000 mg"]
[1, "B01AC06", "ACIDO ACETILSALICILICO", "CARDIOASPIRIN*30 cpr gastrores 100 mg"]
[1, "B01AB05", "ENOXAPARINA SODICA", "CLEXANE*6 siringhe 4.000 UI 0,4 ml"]
[1, "B01AB05", "ENOXAPARINA SODICA", "CLEXANE*6 siringhe 2.000 UI 0,2 ml"]
[1, "C08CA01", "AMLODIPINA BESILATO", "NORVASC*14 cpr 10 mg"]
[1, "C09DA04", "IRBESARTAN/IDROCLOROTIAZIDE", "COAPROVEL*28 cpr riv 300 mg + 25 mg"]
[1, "N02AA55", "OXICODONE CLORIDRATO/NALOXONE CLORIDRATO DIIDRATO", "TARGIN*28 cpr riv 5 mg + 2,5 mg rilascio prolungato"]
[1, "A10BA02", "METFORMINA CLORIDRATO", "METFORALMILLE*60 cpr riv 1.000 mg"]
[1, "B01AB05", 

{#<CommercialDrug @values={:id=>28531022, :aic_code=>"028531022", :ean_code=>nil, :emea_code=>nil, :pharmaceutical_company_code=>nil, :abbreviated_name=>"TRIATEC HCT*14CPR 5MG+25MG", :abbreviated_brand_name=>"TRIATEC HCT", :name=>"TRIATEC HCT*14 cpr 5 mg + 25 mg", :brand_name=>"TRIATEC HCT", :manufacturer__pharmaceutical_company_id=>631709991, :authorization_holder__pharmaceutical_company_id=>nil, :atc_class_id=>1167084536, :pharmaceutical_product_id=>900513, :authorization_date=>#<Date: 1994-11-15 ((2449672j,0s,0n),+0s,2299161j)>, :marketing_date=>#<Date: 1996-04-10 ((2450184j,0s,0n),+0s,2299161j)>, :suspension_date=>nil, :suspension_reason=>nil, :suspension_source=>nil, :suspension_source_act_nbr=>nil, :suspension_source_act_year=>nil, :remarketing_date=>nil, :remarketing_reason=>nil, :remarketing_source=>nil, :remarketing_source_act_nbr=>nil, :remarketing_source_act_year=>nil, :withdrawal_date=>nil, :withdrawal_reason=>nil, :withdrawal_source=>nil, :withdrawal_source_act_nbr=>nil, :

## PRR

In [ ]:
r = Adr[10030109].drug_count start_date: Date.today - (365*10),  atc_prefix: "B01AA03"

r.to_a
.map{|k,v| [ {name: k[0].name, atc: k[1]} , v ] }.to_h
.each{|k,v| puts "#{k} => #{v}"} 
nil

In [ ]:
{:name=>"ENOXAPARINA SODICA", :atc=>"B01AB05"} => 3
{:name=>"ATORVASTATINA CALCIO TRIIDRATO", :atc=>"C10AA05"} => 2
{:name=>"SITAGLIPTIN FOSFATO MONOIDRATO/METFORMINA CLORIDRATO", :atc=>"A10BD07"} => 1
{:name=>"ACIDO ACETILSALICILICO", :atc=>"B01AC06"} => 1
{:name=>"WARFARIN SODICO", :atc=>"B01AA03"} => 4
{:name=>"FONDAPARINUX SODICO", :atc=>"B01AX05"} => 1
{:name=>"RAMIPRIL/IDROCLOROTIAZIDE", :atc=>"C09BA05"} => 1
{:name=>"OXICODONE CLORIDRATO/NALOXONE CLORIDRATO DIIDRATO", :atc=>"N02AA55"} => 1
{:name=>"AMLODIPINA BESILATO", :atc=>"C08CA01"} => 1
{:name=>"IRBESARTAN/IDROCLOROTIAZIDE", :atc=>"C09DA04"} => 1
{:name=>"METFORMINA CLORIDRATO", :atc=>"A10BA02"} => 1
{:name=>"RISPERIDONE", :atc=>"N05AX08"} => 1
{:name=>"PROMAZINA CLORIDRATO", :atc=>"N05AA03"} => 1
{:name=>"METOTREXATO SODICO", :atc=>"L04AX03"} => 1
{:name=>"GEMCITABINA CLORIDRATO", :atc=>"L01BC05"} => 1
{:name=>"METOCLOPRAMIDE MONOCLORIDRATO MONOIDRATO", :atc=>"A03FA01"} => 1
{:name=>"LENALIDOMIDE", :atc=>"L04AX04"} => 1
{:name=>"BISOPROLOLO EMIFUMARATO", :atc=>"C07AB07"} => 2
{:name=>"ROSUVASTATINA SALE DI CALCIO", :atc=>"C10AA07"} => 1
{:name=>"RAMIPRIL", :atc=>"C09AA05"} => 1
{:name=>"ACIDO ACETILSALICILICO/MAGNESIO IDROSSIDO/ALGELDRATO", :atc=>"B01AC06"} => 1
{:name=>"FERROSO SOLFATO", :atc=>"B03AA07"} => 1
{:name=>"SODIO FERRIGLUCONATO", :atc=>"B03AB"} => 1
{:name=>"NIFEDIPINA", :atc=>"C08CA05"} => 1
{:name=>"PRIDINOLO MESILATO", :atc=>"M03BX03"} => 1

## Query example

In [ ]:
Treatment.all
.select{|t| not t.mpd_medicinal_product_id.nil? }
.group_by{|t| t.mpd_medicinal_product_id}
.to_a.map{|k, items| [k, items.length] }
.select{|_, count| count >= 1000 }

In [ ]:
[[12,12], [12,10], [23, 56],[14, 55], [23, 23], [23,78]]
.group_by{|item| item[0]}
.to_a.map{|k, items| [k, items.length] }

In [ ]:
a = [[12, [2, 67]], [23, 3], [14, 1]]

In [ ]:
a.flatten

In [ ]:
a.flatten(1)

In [ ]:
{a: 23, b:67 }.to_a.to_h